In [11]:
import errno
import os
from pathlib import Path

import pandas as pd
from ipyfilechooser import FileChooser
# import matplotlib.pyplot as plt

Константы с параметрами и заголовками

In [2]:
# Параметры из файлов с данными которые нужны
NEEDED_PARAMETERS = (
    "hmax",
    "FiltWindow",
    "Time Range",
    "Minimum duration of series",
    "time step"
)
# Заголовки колонок данных в текстовом файле
DATA_HEADERS = (
    "tsn", "time", "el", "az",
    "latp", "lonp", "tec",
    "tec_filtered", "validity"
)

In [3]:
def get_params_datafile(filename=""):
    """Функция для сбора параметров из файла .dat
    из строк в начале файла начинающихся с символа #.
    Возвращает словарь.
    """
    if not filename:
        raise ValueError("Filename is needed.")

    if not os.path.isfile(filename):
        raise FileNotFoundError(
            errno.ENOENT,
            os.strerror(errno.ENOENT),
            filename
        )

    params = []

    with open(filename, 'r') as tec_file:
        for line in tec_file.readlines():
            if line.strip().startswith("#"):  # это параметр
                param = line.strip()[1:]
                param = param.split("=")
                if len(param) != 2:  # заголовки пропускаем
                    continue
                param = tuple(el.strip() for el in param)
                params.append(param)
    
    params = dict(params)

    # оставляем только нужные параметры
    return {
        key: params[key]
        for key
        in NEEDED_PARAMETERS
    }

In [4]:
def get_dataframe_datafile(filename=""):
    """Возвращает датафрейм с данными из текстового файла TEC.
    Заголовки задаются в константе."""
    return pd.read_csv(
        filepath_or_buffer=filename,
        delim_whitespace=True,  # разделитель - пробел(ы)
        comment="#",  # пропускаем комменты
        names=DATA_HEADERS, # задание заголовков из списка
    )

In [18]:
# Create and display a FileChooser widget
fc = FileChooser('.')
display(fc)

# Change defaults and reset the dialog
fc.default_path = '.'
fc.reset()

# Change hidden files
fc.show_hidden = False

# Show or hide folder icons
fc.use_dir_icons = True

# Set multiple file filter patterns (uses https://docs.python.org/3/library/fnmatch.html)
fc.filter_pattern = ['*.dat', '*.txt']

# Change the title (use '' to hide)
fc.title = '<b>Выберите файл с данными</b>'

# Sample callback function
# def change_title(chooser):
#     chooser.title = '<b>Callback function executed</b>'

# Register callback function
# fc.register_callback(change_title)

FileChooser(path='.', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='False',…

In [20]:
# Print the selected path, filename, or both
print(fc.selected)

/home/alex/Yandex.Disk/Аспирантура/my research/TEST_TEC_DATA/165/G20/stk2G20_165.dat


In [21]:
params = get_params_datafile(fc.selected)

In [22]:
params

{'hmax': '280',
 'FiltWindow': '60 :    300 seconds',
 'Time Range': '0.000 : 24.000',
 'Minimum duration of series': '10 minutes',
 'time step': '30'}

In [23]:
dataframe = get_dataframe_datafile(fc.selected)

In [24]:
print(dataframe)


      tsn     time       el         az     latp      lonp       tec  \
0     764   6.3667  7.93273  322.99857  51.5037  131.7034  12.01332   
1     765   6.3750  8.00573  322.82059  51.4508  131.7142  12.00824   
2     766   6.3833  8.07787  322.64206  51.3985  131.7244  12.02581   
3     767   6.3917  8.14913  322.46298  51.3467  131.7340  12.00263   
4     768   6.4000  8.21952  322.28339  51.2955  131.7431  12.06604   
..    ...      ...      ...        ...      ...       ...       ...   
870  2342  19.5167  7.78179   64.26305  47.2919  155.9603   6.10383   
871  2343  19.5250  7.60772   64.36535  47.3014  156.1141   6.11463   
872  2344  19.5333  7.43388   64.46795  47.3105  156.2699   6.16158   
873  2345  19.5417  7.26027   64.57084  47.3193  156.4275   6.11470   
874  2346  19.5500  7.08687   64.67403  47.3278  156.5871   6.23488   

     tec_filtered  validity  
0         0.00000         3  
1        -0.00239         0  
2         0.00083         0  
3        -0.01064         0

In [25]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [26]:
dataframe.plot(x="time", y="tec_filtered")

<AxesSubplot:xlabel='time'>